<a href="https://colab.research.google.com/github/superoverflow/notebooks/blob/main/random_num_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import plotly.express as px
import numpy as np
from functools import reduce

def random_but_greater_than(previous_value: float, bottom_line: float) -> float:
  new_value = previous_value + np.random.randn()
  return max(new_value, bottom_line)


size = 1024
m = 0.02

x = np.arange(0, size)
min_y = m * x

y = reduce(lambda a, b: [*a, max(a[-1] + np.random.randn(), m * b)], x, [0])
y = np.delete(y, 0)

fig = px.scatter(x = x, y = y)
fig.add_traces(px.line(x=x, y=min_y, color_discrete_sequence=["red"]).data)
fig.show()